# Monthly FAPAR Composite Calculation

This notebook calculates monthly FAPAR (Fraction of Absorbed Photosynthetically Active Radiation) composites by combining 8-day FAPAR data. The process includes loading and masking data, calculating weighted values based on date overlap, and saving monthly composites.


In [ ]:
import os
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
from rasterio.enums import Resampling
from datetime import datetime, timedelta
from calendar import monthrange


## Paths - Set These Paths Before Running

In [ ]:
# Main directory containing the project files
main_dir = 'path_to_main_directory'
# Directory containing 8-day FAPAR data clipped to the Area of Interest (AOI)
fapar_glass_dir = os.path.join(main_dir, 'Clipped_FAPAR')
# Example Copernicus FAPAR file for reference (shape and CRS)
copernicus_example_path = os.path.join(main_dir, 'Copernicus_FAPAR_example.tif')
# Directory containing the boundary shapefile for masking
boundary_shapefile_dir = os.path.join(main_dir, 'Boundary')


## Load and Mask Shapefile

In [ ]:
def load_shape_file(filepath):
    """Load a shapefile for creating a mask over the AOI."""
    shpfile = gpd.read_file(filepath)
    print("Shapefile loaded.")
    return shpfile

def select_shape(shpfile):
    """Create a union of selected shapes for masking."""
    col_code = 'ISO3_CODE'
    country_codes = ['ZAF', 'LSO', 'SWZ']
    selected_rows = shpfile[shpfile[col_code].isin(country_codes)]
    mask_polygon = gpd.GeoDataFrame(geometry=[selected_rows.geometry.unary_union])
    print("Mask created.")
    return mask_polygon

# Load shapefile and create mask
shapefile_path = os.path.join(boundary_shapefile_dir, 'boundary_shapefile.shp')
shapefile = load_shape_file(shapefile_path)
mask_polygon = select_shape(shapefile)

# Load CRS from example file
copernicus_ex = rxr.open_rasterio(copernicus_example_path, masked=True)
mask_polygon.crs = copernicus_ex.rio.crs


## Calculate Weights Based on Overlapping Days

In [ ]:
def weights_start(start_date):
    """Calculate weight for an 8-day FAPAR period starting in a given month."""
    num_days = monthrange(start_date.year, start_date.month)[1]
    day_of_month = start_date.day
    diff = (num_days + 1) - day_of_month
    day_obs = min(diff, 8)
    return day_obs / num_days

def weights_end(end_date):
    """Calculate weight for an 8-day FAPAR period ending in a given month."""
    num_days = monthrange(end_date.year, end_date.month)[1]
    return end_date.day / num_days


## Process FAPAR Data for Each Month and Year

In [ ]:
def calculate_monthly_composites():
    """Calculate monthly FAPAR composites from 8-day FAPAR data over multiple years."""
    months = ['Jan', 'Feb', 'March', 'April', 'May', 'June', 'July', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    years = sorted(os.listdir(fapar_glass_dir))
    count = 0

    for month in range(1, 13):
        for year in years:
            month_data = []
            for image_file in os.listdir(os.path.join(fapar_glass_dir, year)):
                y = image_file[16:20]
                doy = image_file[20:23].rjust(3, '0')
                start_date = datetime.strptime(f"{y}-{doy}", "%Y-%j")
                end_date = start_date + timedelta(days=8)

                if start_date.month == month:
                    weight = weights_start(start_date)
                elif end_date.month == month and end_date.day <= 7:
                    if month == 1 or end_date.month == 1:
                        continue
                    weight = weights_end(end_date)
                else:
                    continue

                image_path = os.path.join(fapar_glass_dir, year, image_file)
                fapar = rxr.open_rasterio(image_path, masked=True)
                fapar *= fapar.attrs.get('scale_factor', 1)  # Apply scale factor if present
                month_data.append(weight * fapar)

            if month_data:
                monthly_composite = sum(month_data).rio.reproject_match(copernicus_ex, resampling=Resampling.nearest)
                monthly_composite_path = os.path.join(main_dir, 'Monthly_FAPAR_Composites', f"{year}_{months[count]}.tif")
                monthly_composite.rio.to_raster(monthly_composite_path)

        count += 1

# Run the function to process FAPAR data
calculate_monthly_composites()
